# <strong><font color=#A52A2A></span> NLP_HW4 </strong>
Dr Asgari - Spring 2023
<br>
98101566 -> Mohammadreza Daviran
<br>
98170668 -> Sara Azarnoush
<br>
98171007 -> Nona Ghazizadeh
<br>
<strong><font color=#A52A2A>Token classification with transformers

# <font color='red'> Table of content </font>
-------

 - Intoduction
 - Transformers
    - Requirements
    - Load Dataset
    - Preprocess
    - Model
        - Requirements
        - Fine Tune
    - Evaluation
        - EM and F1
        - Other Metrics
 - Inference

# <font color='red'> Introduction </font>
-------

In this project, we Implement a model for extractive question and answer.

It inputs a text and question and returns an answer from the text.

We import project requirements and download the SajjadAyoubi/persian_qa dataset and preprocess it and save the preprocessed data.

We make labels from the start index and end index of the answer in text and use HuggingFace pretrained model to vectorize them and predict the answers for the related questions.

More information is available in each section.

# <font color='red'> Transformer </font>
----
Transformer is a new architecture that aims to solve sequence-to-sequence tasks while easily handling long-distance dependencies. Computing the input and output representations without using sequence-aligned RNNs or convolutions and it relies entirely on self-attention. In this matter we use hugging face pretrained model and we get a fine-tuned model according to our dataset from that.

## <font color='green'> Requirements

In [1]:
!pip install transformers datasets evaluate

In [1]:
from datasets import load_dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, pipeline, DefaultDataCollator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import pandas as pd

/home/amir/.venvs/env38gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-07 03:35:51.874245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 03:35:51.961647: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-07 03:35:51.981720: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to regi

## <font color='green'> Load Dataset </font>

There are always some redundant and inaccurate samples in the data. After loading the dataset we omit these samples from our training and test data. These samples doesn't have any answer to the related question so they are actually useless!

In [3]:
dataset = load_dataset("SajjadAyoubi/persian_qa")
dataset["train"] = dataset["train"].filter(lambda x: x["answers"]["answer_start"])
dataset["validation"] = dataset["validation"].filter(lambda x: x["answers"]["answer_start"])
dataset

Using the latest cached version of the module from /home/amir/.cache/huggingface/modules/datasets_modules/datasets/SajjadAyoubi--persian_qa/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225 (last modified on Fri Jul  7 02:57:31 2023) since it couldn't be found locally at SajjadAyoubi/persian_qa., or remotely on the Hugging Face Hub.
Found cached dataset persian_qa (/home/amir/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225)
100%|██████████| 2/2 [00:00<00:00, 956.84it/s]
Loading cached processed dataset at /home/amir/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225/cache-a9db7a9b721383a3.arrow
Loading cached processed dataset at /home/amir/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225/cache-20773be1cb7b6c5c.arro

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 6306
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 651
    })
})

## <font color='green'> Preprocess </font>

The preprocess_data function helps us to preprocess the input data for Extractive Question Answering by encoding the questions and context, determining the start and end positions of the answers within the encoded inputs, and adding these positions to the encoded inputs. This preprocessing is typically done to prepare the data for training and inference in an Extractive Question Answering HuggingFace model that expects specific input formats.

In [6]:
def preprocess_data(data):
    processed_questions = [question.strip() for question in data["question"]]
    tokenizer = AutoTokenizer.from_pretrained("ForutanRad/bert-fa-QA-v1")
    encoded_inputs = tokenizer(
        processed_questions,
        data["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = encoded_inputs.pop("offset_mapping")
    answers = data["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = encoded_inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > end_char or offsets[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    encoded_inputs["start_positions"] = start_positions
    encoded_inputs["end_positions"] = end_positions
    return encoded_inputs

In [7]:
tokenized_processed_data = dataset.map(preprocess_data, batched=True,remove_columns=dataset["train"].column_names)

Loading cached processed dataset at /home/amir/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225/cache-5282968ad65a3844.arrow
Loading cached processed dataset at /home/amir/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225/cache-d95b5e89c5ade0b8.arrow


## <font color='green'> Model </font>

After preprocessing the data, we use AutoModelForQuestionAnswering from "ForutanRad/bert-fa-QA-v1". Then we fine tune the model by training it on the tokenized data.

In [8]:
model = AutoModelForQuestionAnswering.from_pretrained("ForutanRad/bert-fa-QA-v1")

### <font color='blue'> Requirements </font>

These are some requirements to help us train the model:

In [9]:
!pip install accelerate -U

In [10]:
!pip install transformers[torch] accelerate

In [11]:
model= model.to('cpu')

### <font color='blue'> Fine Tune </font>

After loading the model, we define the training arguments and configure the Trainer class to train a new model. The TrainingArguments object (args) sets various hyperparameters and settings for training, while the Trainer object (trainer) encapsulates the training process and takes care of executing the training loop. Finally, trainer.train() begins the training process using the specified model, datasets, and training configuration.

In [12]:
args = TrainingArguments(
    output_dir="qa_answering_fine_tuned",
    evaluation_strategy="epoch", learning_rate=2e-6,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    num_train_epochs=5, weight_decay=0.01,)

trainer = Trainer(
    model=model, args=args,
    train_dataset=tokenized_processed_data["train"], eval_dataset=tokenized_processed_data["validation"],
    tokenizer=AutoTokenizer.from_pretrained("ForutanRad/bert-fa-QA-v1"), data_collator=DefaultDataCollator(),)

trainer.train()

/home/amir/.venvs/env38gpu/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  6%|▋         | 500/7885 [02:42<31:02,  3.97it/s]  

{'loss': 1.1214, 'learning_rate': 1.873176918199112e-06, 'epoch': 0.32}


 13%|█▎        | 1000/7885 [04:50<28:59,  3.96it/s] 

{'loss': 1.0584, 'learning_rate': 1.7463538363982244e-06, 'epoch': 0.63}


 19%|█▉        | 1500/7885 [06:59<27:15,  3.90it/s]  

...

 95%|█████████▌| 7500/7885 [34:14<01:39,  3.87it/s]

{'loss': 0.6701, 'learning_rate': 9.765377298668358e-08, 'epoch': 4.76}


                                                   
100%|██████████| 7885/7885 [36:07<00:00,  3.64it/s]

{'eval_loss': 2.4652397632598877, 'eval_runtime': 11.0729, 'eval_samples_per_second': 58.792, 'eval_steps_per_second': 14.721, 'epoch': 5.0}
{'train_runtime': 2167.1487, 'train_samples_per_second': 14.549, 'train_steps_per_second': 3.638, 'train_loss': 0.8638666804863367, 'epoch': 5.0}


TrainOutput(global_step=7885, training_loss=0.8638666804863367, metrics={'train_runtime': 2167.1487, 'train_samples_per_second': 14.549, 'train_steps_per_second': 3.638, 'train_loss': 0.8638666804863367, 'epoch': 5.0})

We save our new model to prevent further model training and cell runnings, because the training process was so overwhelming and time-consuming.

In [14]:
trainer.save_model("QA_model")model= model.to('cpu')

In [9]:
!zip Transformer_Model.zip Transformer_Model/*

## <font color='green'> Evaluation </font>

From now on, we evaluate our model by different metrics.

In [14]:
model= model.to('cpu')

In [15]:
question_answerer = pipeline("question-answering", model=model, tokenizer=AutoTokenizer.from_pretrained("ForutanRad/bert-fa-QA-v1"))
question_answerer(question=question, context=context)

{'score': 0.9891430139541626, 'start': 61, 'end': 67, 'answer': 'مادرید'}

In the following loop, we gather all the predictions for the test data with our fine-tuned model.

In [16]:
model = model.to('cpu')
predictions = []
for i in tqdm(range(len(dataset["validation"]))):
    question = dataset["validation"][i]["question"]
    context = dataset["validation"][i]["context"]
    prediction = question_answerer(question=question, context=context)
    predictions.append(prediction)

100%|██████████| 651/651 [01:59<00:00,  5.47it/s]


In [17]:
model_predictions = [{"prediction_text": pred['answer'], "id": str(i)} for i, pred in enumerate(predictions)]
model_predictions[:10]

[{'prediction_text': 'مادرید', 'id': '0'},
 {'prediction_text': 'فیفا', 'id': '1'},
 {'prediction_text': '۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی',
  'id': '2'},
 {'prediction_text': '«سلطنتی»', 'id': '3'},
 {'prediction_text': 'مادرید، پایتخت اسپانیا', 'id': '4'},
 {'prediction_text': 'شاه آلفونسو سیزدهم', 'id': '5'},
 {'prediction_text': 'بارسلونا', 'id': '6'},
 {'prediction_text': 'باشگاه فوتبال بارسلونا که بیشتر بارسلونا یا به صورت مخفف بارسا شناخته می\u200cشود',
  'id': '7'},
 {'prediction_text': 'اسپانیایی', 'id': '8'},
 {'prediction_text': 'منطقهٔ کاتالونیای اسپانیا', 'id': '9'}]

In [18]:
actual_answers = [{"answers": ref["answers"], "id": str(i)} for i, ref in enumerate(dataset["validation"])]
actual_answers[:10]

[{'answers': {'text': ['مادرید', 'مادرید'], 'answer_start': [19, 19]},
  'id': '0'},
 {'answers': {'text': ['فیفا', 'به انتخاب فیفا'], 'answer_start': [218, 208]},
  'id': '1'},
 {'answers': {'text': ['۱۳', '۱۳'], 'answer_start': [329, 329]}, 'id': '2'},
 {'answers': {'text': ['سلطنتی', 'سلطنتی'], 'answer_start': [648, 648]},
  'id': '3'},
 {'answers': {'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا'],
   'answer_start': [61, 76]},
  'id': '4'},
 {'answers': {'text': ['شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد',
    'شاه آلفونسو سیزدهم'],
   'answer_start': [672, 672]},
  'id': '5'},
 {'answers': {'text': ['رقیب اصلی این تیم نیز، بارسلونا است', 'بارسلونا'],
   'answer_start': [857, 880]},
  'id': '6'},
 {'answers': {'text': ['بارسلونا', 'باشگاه فوتبال بارسلونا'],
   'answer_start': [14, 0]},
  'id': '7'},
 {'answers': {'text': ['اسپانیا', 'اسپانیا'], 'answer_start': [103, 103]},
  'id': '8'},
 {'answers': {'text': ['منطقهٔ کاتالونیای', 'کاتالونیا'],
   'answer_start': [138, 145]},
  

### <font color='blue'> EM and F1 </font>


from "evaluate" we have imported "load" which ables us to use squad_metric. This metric can calculate EM and F1 score for us.

In [23]:
squad_metric = load("squad")

In [24]:
results = squad_metric.compute(predictions=model_predictions, references=actual_answers)
print("Exact Matching metric: ", results['exact_match'])
print("f1 score: ", results['f1'])

Exact Matching metric:  43.4715821812596
f1 score:  68.81148674323983


### <font color='blue'> Other Metrics </font>

As a matter of fact, for this task and with this model it's not necessary to calculate recall ot precision score but we have calculate them. Albeit, these results have no information at all and we have to use EM and f1_score to evaluate the model.

In [19]:
test_lables = dataset["validation"]["answers"]
test_lables[0]

{'text': ['مادرید', 'مادرید'], 'answer_start': [19, 19]}

In [20]:
predictions[0]['answer']

'مادرید'

In [21]:
ground_truth = []
plain_predictions = []
for i in range(len(predictions)):
  ground_truth.append(test_lables = dataset["validation"]["answers"]
predictions[0]['answer']test_lables[0][i]['text'][0])
  plain_predictions.append(predictions[i]['answer'])
print(plain_predictions[10])
print(ground_truth[10])

نهادی برای ترویج فرهنگ کاتالان و کاتالانیسم است
به نوعی، نهادی برای ترویج فرهنگ کاتالان و کاتالانیسم


In [22]:
print("accuracy: ", accuracy_score(ground_truth, plain_predictions))
print("precision_macro: ", precision_score(ground_truth, plain_predictions, average='macro'))
print("precision_micro: ", precision_score(ground_truth, plain_predictions, average='micro'))
print("recall_macro: ", recall_score(ground_truth, plain_predictions, average='macro'))
print("recall_micro: ", recall_score(ground_truth, plain_predictions, average='micro'))b
confusion_mat = confusion_matrix(ground_truth, plain_predictions)
confusion_mat_df = pd.DataFrame(confusion_mat)
display(confusion_mat_df)

accuracy:  0.2903225806451613
precision_macro:  0.17105263157894737
precision_micro:  0.2903225806451613
recall_macro:  0.17434210526315788
recall_micro:  0.2903225806451613
f1_macro:  0.1716791979949875
f1_micro:  0.2903225806451613


/home/amir/.venvs/env38gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/amir/.venvs/env38gpu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1,2,3,4,5,6,7,8,9,...,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# <font color='red'> Inference </font>

As it was expected before, this model worked out so much better than the implemented HMM model on this task. First of all pretrained models have been trained on a lot of data and even by fine tuning them we can get better results! However for HMM models and other simple implemented models, there aren't a lot of data to use so we are restrained to use just our training data so we don't have to expect great results. Besides, the Transformers are way better models than the HMM and LSTM/CRF models for sequence-to-sequence tasks like EQA. Overall we can tell that the scores for transformer model is higher and better than the same thing for other models on the EQA task!